# Notebook C: Find GSM flux bounds by constraining the GSM with 13C-MFA
This Jupyter notebook focuses on using Flux Variability Analysis (FVA) to find the bounds of each reaction in a Genome Scale Model (GSM) 
when 13C-MFA reactions are used for GSM boundries.

### Objective
The primary objective of this analysis is to use the GSM to extrapolate the rates of reactions outside the scope of 13C-MFA. We explore the genome scale metabolic flux profiles when Y. lipolytica consumes glucose, glycerol, acetate, and oleic acid. 

### Important Functions
- `load_and_preprocess_data`: Loads genome-scale models and 13C-MFA data, and performs necessary preprocessing.
- `perform_CFBA`: Executes Constrained Flux Balance Analysis and prepares the data for comparison.
- `compare_flux_analysis`: Compares CFBA results with 13C-MFA data, highlighting key similarities and differences.
- `analyze_carbon_sources`: Analyzes the impact of different carbon sources on metabolic fluxes within the CFBA model.
- `integrate_strain_design`: Integrates StrainDesign predictions to enhance the understanding of the metabolic model.


### Load imports

In [1]:
import cobra
import straindesign as sd
import pandas as pd
import sys


source_dir = '../src'
sys.path.append(source_dir)
from add_flux_column_to_13c_flux_df import add_flux_column_to_13c_flux_df
from add_fva_columns_to_13c_flux_df import add_fva_columns_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot
from make_rxn_constraint_string import make_rxn_constraint_string

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model('../genome_scale_models/iYLI647_corr_3.json')

### Rename some reactions to remove parentheses
This is because parentheses in reaction ids cause problems with StrainDesign

In [3]:
model.reactions.get_by_id('EX_glc(e)').id = 'EX_glc_e'
model.reactions.get_by_id('EX_glyc(e)').id = 'EX_glyc_e'
model.reactions.get_by_id('EX_ocdcea(e)').id = 'EX_ocdcea_e'
model.reactions.get_by_id('EX_h2o(e)').id = 'EX_h2o_e'
model.reactions.get_by_id('EX_h(e)').id = 'EX_h_e'
model.reactions.get_by_id('EX_nh4(e)').id = 'EX_nh4_e'
model.reactions.get_by_id('EX_o2(e)').id = 'EX_o2_e'
model.reactions.get_by_id('EX_pi(e)').id = 'EX_pi_e'
model.reactions.get_by_id('EX_so4(e)').id = 'EX_so4_e'

# print an example reaction
model.reactions.get_by_id('EX_glc_e')

Reaction identifier,EX_glc_e
Name,D Glucose exchange
Memory address,0x1639456f0
Stoichiometry,glc_D[e] <=> D_Glucose <=>
GPR,YALI0D01111g or YALI0D18876g or YALI0D00132g or YALI0B01342g or YALI0E23287g or YALI0B00396g or...
Lower bound,-10.0
Upper bound,1000.0


### Load 13C-MFA data

In [4]:
central_rxn_df = pd.read_excel('../data/13c_mfa/INCA_12082023_GR.xlsx', sheet_name='new mfa from gsm_1208')

# calculate the number of reactions in the 13C MFA that are mapped to the GSM
mapped_rxn_df = central_rxn_df.dropna(subset = ["reaction_ids"])

print(f'There are {len(mapped_rxn_df)} reactions in the 13C MFA that are mapped to the GSM')

central_rxn_df.head()

There are 43 reactions in the 13C MFA that are mapped to the GSM


/Users/garrettroell/yarrowia_eflux2/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Unnamed: 0.1,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,gluc 1208,gluc LB,gluc UB,...,glycerol_GSM_LB,glycerol_GSM_UB,glycerol_mfa_bound_feasibility,OA 1208 (6),OA LB,OA UB,oleic_acid_GSM_flux,oleic_acid_GSM_LB,oleic_acid_GSM_UB,oleic_acid_mfa_bound_feasibility
0,0,uptake,uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,substrate_uptake,cytosol,NaN,NaN,NaN,...,0.00000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
1,1,uptake,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,NaN,NaN,...,100.00000,100.000000,fully feasible,NaN,NaN,NaN,0.000000,-37.261931,0.000000,NaN
2,2,uptake,R3 dhap net,Glyc3P <-> DHAP + UQH2,NaN,emp,cytosol,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,NaN,...,0.00000,0.000000,NaN,100,100.0000,100.0000,100.000000,100.000000,100.000000,NaN
4,4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.4581,10.7637,19.8795,...,-96.59923,-3.061031,fully feasible,-315.0748,-328.5227,-268.7251,-27.756662,-510.795313,-24.703706,fully feasible


### Define a function to build constraint strings from 13C-MFA data

In [7]:
def build_13c_constraint_string(central_rxn_df, lower_bound_row, upper_bound_row):
    
    # key node reactions to constrain
    key_pathway_reactions = {
        'EMP': 'GAPD',
        'TCA': 'CSm',
        'PPP': 'GND',
    }
    
    constraint_strings = []

    for pathway, reaction_ids in key_pathway_reactions.items():
        row = central_rxn_df.loc[central_rxn_df['reaction_ids'] == reaction_ids]
        # get the lower bound for this reaction
        lower_bound = row[lower_bound_row]
        upper_bound = row[upper_bound_row]

        constraint_string = make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound)
        constraint_strings.append(constraint_string)

    full_constraint_string = ', '.join(constraint_strings)
    return full_constraint_string

    

build_13c_constraint_string(central_rxn_df, 'gluc LB', 'gluc UB')


'GAPD >= 8    129.5795\nName: gluc LB, dtype: float64, GAPD <= 8    136.7617\nName: gluc UB, dtype: float64, CSm >= 22    45.268\nName: gluc LB, dtype: float64, CSm <= 22    63.8681\nName: gluc UB, dtype: float64, GND >= 13    61.5053\nName: gluc LB, dtype: float64, GND <= 13    69.6279\nName: gluc UB, dtype: float64'

# Glucose

### Make constraint string from 13C-MFA data for glucose

In [20]:
# can build this to include
mfa_reactions_to_use = [
  'G6P <-> F6P', # start of gylcolysis
  'G6P -> PG6 + NADPH', # start of pentose phosphate pathway
  'PEP -> PYRcyt + ATP', # end of glycolysis
  'ACCOAmit + OAAmit -> CITmit', # start of TCA cycle
]

constraint_strings = []

for mfa_reaction in mfa_reactions_to_use:
    # get the reaction row
    reaction_row = central_rxn_df[central_rxn_df['Equation'] == mfa_reaction]

    # get reaction IDs
    reaction_ids = reaction_row['reaction_ids'].values[0]

    # get the glucose lower bound
    glucose_lb = reaction_row['gluc LB'].values[0]

    # get the glucose upper bound
    glucose_ub = reaction_row['gluc UB'].values[0]

    # generate a constraint string
    constraint_string = make_rxn_constraint_string(reaction_ids, glucose_lb, glucose_ub)

    # add the constraint string to the list
    constraint_strings.append(constraint_string)

glucose_constraint_string = ', '.join(constraint_strings)

glucose_constraint_string

'PGI >= 10.7637, PGI <= 19.8795, G6PDH2 >= 61.5053, G6PDH2 <= 69.6279, PYK >= 144.1458, PYK <= 463.2564, CSm >= 45.268, CSm <= 63.8681'

### Run Glucose pFBA with MFA constraints

In [21]:
# update the media to minimal medium with glucose as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

glucose_fba_solution = sd.fba(model, constraints=glucose_constraint_string, obj='biomass_glucose', obj_sense='maximize', pfba=1)

print()

max_glucose_biomass_flux = glucose_fba_solution['biomass_glucose']
PGI_flux = glucose_fba_solution['PGI']
print(f'Maximum biomass flux: {max_glucose_biomass_flux}.')
print(f'PGI flux: {PGI_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glucose_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glucose_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glucose_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glucose_gsm_fba_df = glucose_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glucose_gsm_fba_df = glucose_gsm_fba_df.drop(columns=['absolute_flux'])

glucose_gsm_fba_df.head()

100 EX_glc_e
10000 EX_h2o_e
10000 EX_h_e
10000 EX_nh4_e
10000 EX_o2_e
10000 EX_pi_e
10000 EX_so4_e

Maximum biomass flux: 12.454101522181.
PGI flux: 19.8795.
The number of active reactions in pFBA: 283


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-597.229160
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],356.689025
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],355.514995
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,351.636016
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-328.343602


### Run Glucose FVA with MFA constraints

In [22]:
# run FVA for 90% of biomass production on the GSM

glucose_fva_solution = sd.fva(
  model, 
  constraints=glucose_constraint_string,
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glucose_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glucose_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glucose_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glucose_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glucose_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glucose_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glucose_gsm_fba_df.to_csv('../results/gsm_fluxes/glucose_gsm_13C_fba.csv', index=False)

# display updated dataframe
glucose_gsm_fba_df


The number of active reactions in FVA: 338


,reaction_id,reaction_name,full_reaction,flux,fva_upper_bound,fva_lower_bound
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-597.229160,-0.0,0.0
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],356.689025,-0.0,0.0
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],355.514995,-0.0,0.0
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,351.636016,1000.0,0.0
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-328.343602,-0.0,0.0
...,...,...,...,...,...,...
490,FA140ACPtm,fatty acyl ACP mitochondrial transport,myrsACP[m] --> myrsACP[c],0.000000,-0.0,0.0
489,FA140ACPH,fatty acyl ACP hydrolase,h2o[c] + myrsACP[c] <=> ACP[c] + h[c] + ttdca[c],0.000000,-0.0,0.0
488,FA120tp,fatty acid peroxisomal transport,ddca[c] --> ddca[x],0.000000,-0.0,0.0
487,FA120ACPtm,fatty acyl ACP mitochondrial transport,ddcaACP[m] --> ddcaACP[c],0.000000,-0.0,0.0


### Build glucose constraint string from 13C-MFA data

In [ ]:
# update the media to minimal medium with glucose as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

lower_bound_row = 'glucose_LB'
upper_bound_row = 'glucose_UB'

constraint_strings = []

# loop over rows in the central flux dataframe
for _, row in central_rxn_df.iterrows():  
    # get the GSM reaction mapping for this reaction
    reaction_ids = row['reaction_ids']
    pathway = row['pathway']

    # get the lower bound for this reaction
    lower_bound = row[lower_bound_row]
    upper_bound = row[upper_bound_row]

    # determine if reaction should be included in the constraint string
    reaction_ids_not_nan = not pd.isna(reaction_ids)
    pathway_not_transport = pathway != 'transport'
    bounds_are_not_nan = not pd.isna(lower_bound) and not pd.isna(upper_bound)
    

    if reaction_ids_not_nan and pathway_not_transport and bounds_are_not_nan:
        print(f'13C-MFA bounds: {lower_bound} - {upper_bound}')

        # make the constraint string
        constraint_string = make_rxn_constraint_string(reaction_ids, lower_bound, upper_bound)

        glucose_fba_solution = sd.fba(model, constraints=constraint_string, obj='biomass_glucose', obj_sense='maximize', pfba=1)

        # add the constraint string to the list
        constraint_strings.append(constraint_string)
        print(constraint_string)
        print(f"max biomass flux: {glucose_fba_solution['biomass_glucose']}")
        print()



full_constraint_string = ', '.join(constraint_strings)
print(len(full_constraint_string))
full_constraint_string 

### Calculate glucose GSM pFBA solution

### Calculate glucose GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glucose_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glc_e = -100.000, biomass_C >= {biomass_fraction * max_glucose_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glucose_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glucose_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glucose_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glucose_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glucose_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glucose_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glucose_gsm_fba_df.to_csv('../results/gsm_fluxes/glucose_gsm_13C_fba.csv', index=False)

# display updated dataframe
glucose_gsm_fba_df


### Add glucose pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, 'glucose_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, f'glucose_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Glycerol

### Calculate glycerol GSM pFBA solution

In [ ]:
# update the media to minimal medium with glycerol as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 100
medium['EX_ocdcea_e'] = 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
glycerol_fba_solution = sd.fba(model, constraints='EX_glyc_e = -100.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_glycerol_biomass_flux = glycerol_fba_solution['biomass_C']
print(f'Maximum biomass flux: {max_glycerol_biomass_flux}.')
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in glycerol_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in glycerol_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': flux,
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
glycerol_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
glycerol_gsm_fba_df = glycerol_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
glycerol_gsm_fba_df = glycerol_gsm_fba_df.drop(columns=['absolute_flux'])

glycerol_gsm_fba_df.head()

### Calculate glycerol GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
glycerol_fva_solution = sd.fva(
  model, 
  constraints=f'EX_glyc_e = -100.000, biomass_C >= {biomass_fraction * max_glycerol_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in glycerol_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in glycerol_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = glycerol_fva_solution.loc[reaction_id, 'maximum']
  lower_bound = glycerol_fva_solution.loc[reaction_id, 'minimum']

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
glycerol_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
glycerol_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
glycerol_gsm_fba_df.to_csv('../results/gsm_fluxes/glycerol_gsm_13C_fba.csv', index=False)

# display updated dataframe
glycerol_gsm_fba_df

### Add glycerol pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, 'glycerol_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, f'glycerol_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Oleic Acid

### Calculate oleic acid GSM pFBA solution

In [ ]:
# update the media to minimal medium with oleic_acid as the sole carbon source
medium = model.medium
medium['EX_glc_e'] = 0
medium['EX_glyc_e'] = 0
medium['EX_ocdcea_e'] = 10 # this prevents overflow
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# print the medium composition
[print(model.medium[m], m) for m in model.medium]

# run biomass-maximizing pFBA
oleic_acid_fba_solution = sd.fba(model, constraints='EX_ocdcea_e = -10.000', obj='biomass_C', obj_sense='maximize', pfba=1)

max_oleic_acid_biomass_flux = oleic_acid_fba_solution['biomass_C']
print(f'Maximum biomass flux: {10 * max_oleic_acid_biomass_flux}.') # restore 100 input flux
print(f'The number of active reactions in pFBA: {sum([abs(flux) > 0.1 for flux in oleic_acid_fba_solution.fluxes.values()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, flux in oleic_acid_fba_solution.fluxes.items():

  reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    'flux': 10 * flux, # restore 100 input flux
    'absolute_flux': abs(flux), # use for sorting, then drop
  })

# make a dataframe from the list of dictionaries
oleic_acid_gsm_fba_df = pd.DataFrame(reactions)

# sort the dataframe by absolute flux
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

# drop the absolute flux column
oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.drop(columns=['absolute_flux'])

oleic_acid_gsm_fba_df.head()

### Calculate oleic acid GSM pFBA FVA 

In [ ]:
# run FVA for 90% of biomass production on the GSM
biomass_fraction = 0.9
oleic_acid_fva_solution = sd.fva(
  model, 
  constraints=f'EX_ocdcea_e = -10.000, biomass_C >= {biomass_fraction * max_oleic_acid_biomass_flux}',
)

# define a function to determine if a reaction is active
def is_active(row):
  return abs(row.maximum) > 0.1 or abs(row.minimum) > 0.1

print(f'The number of active reactions in FVA: {sum([is_active(row) for _, row in oleic_acid_fva_solution.iterrows()])}')

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
fva_upper_bounds = []
fva_lower_bounds = []

# loop over the reactions in the GSM
for _, row in oleic_acid_gsm_fba_df.iterrows():
  reaction_id = row.reaction_id

  # get the upper and lower bounds from the FVA solution
  upper_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'maximum'] # restore 100 input flux
  lower_bound = 10 * oleic_acid_fva_solution.loc[reaction_id, 'minimum'] # restore 100 input flux

  fva_upper_bounds.append(upper_bound)
  fva_lower_bounds.append(lower_bound)

# add the upper and lower bounds to the dataframe
oleic_acid_gsm_fba_df['fva_upper_bound'] = fva_upper_bounds
oleic_acid_gsm_fba_df['fva_lower_bound'] = fva_lower_bounds

# save the dataframe to a csv file
oleic_acid_gsm_fba_df.to_csv('../results/gsm_fluxes/oleic_acid_gsm_13C_fba.csv', index=False)

# display updated dataframe
oleic_acid_gsm_fba_df


### Add oleic acid pFBA columns to 13C-MFA data

In [ ]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_flux_column_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, 'oleic_acid_pFBA_flux')

# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_fva_columns_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, f'oleic_acid_pFBA_{100*biomass_fraction}%')

central_rxn_df.head()

# Save Data

### Save central flux data with pFBA data added

In [ ]:
# save the dataframe to a csv file
central_rxn_df.to_csv('../results/central_fluxes/13C_pfba.csv', index=False, encoding='utf-8-sig')